In [1]:
# Core implementation of turbulent heat and water continuity, we provide the main commands to compute the 
# discrete equations in Algorithms 1 and 2. 

import numpy as np

# Main parameters (please refer to Table 2 in the manuscript)
rho_d = 2500 # dry air density
gass_r = 0.287 # dry air gass constant
gamma0 = -6.5e-3 # lapse rates
gamma1 = 6.5e-3
z1 = 8e3 # inversion altitude
s_k = 0.1 # turbulent scale, this depends on your specific grid s_k = (\Delta x \Delta y \Delta z)^{1/3}
dx = 0.1 # dummy dx for testing purposes
c_e = 0.3 # ventilation coefficient
p_G = 1.013e8 # pressure at ground level
c_p = 1.0035e3 # heat capacity dry air
grav = -9.81 # gravity acceleration
r_dust = 7.5e-5 # radius of dust particle
rho_dust = 490 # dust density
nu_dust = 1.2 # dust viscosity
m_dust = 7.5e-10 # dust mass
c_k = 0.4 # von karman constant
B = 0.5 # Holland's radial pressure
R_max = 150e3 # maximum storm radius
q_v0 = 0.1 # initial moisture

# We save the atmospheric system using five main sets of fields
# \vector{u}, \vector{ud} - fluid and dust velocities
# k - turbulent kinetic energy
# theta - potential temperature
# q_a = (q_v, q_w, q_i, q_r, q_s, q_g) - hydrometeors (vapor, water cloud, ice cloud, rain, snow, graupel)

# Get background temperature profile
# Input:
# groundT - 2D grid with ground temperature (can be time dependent - series of 2D textures)
# idx = (i,j,k) - 3D coordinates
# gamma0, gamma1 - lapse rates
# z1 - inversion altitude
def backgroundT(groundT, idx, gamma0, gamma1, z1):
    tempGround = groundT[idx[0]][idx[1]]
    if(idx[2]<=z1):
        return tempGround - gamma0*idx[2]
    else:
        return tempGround - gamma0*z1 - gamma1* (idx[2]-z1)


# Get background pressure profile
# Input:
# groundT - 2D grid with ground temperature
# x - 3D coordinates
# q_v - mixing ratio of vapor
def backgroundP(groundT, idx, q_v):
    return rho_d * gass_r * backgroundT(groundT, idx, gamma0, gamma1, z1) * (1.0+0.61*q_v)

# Get turbulent viscosity
# Input:
# k - turbulent energy field
# idx  = (i,j,k) - 3D coordinates
def nu_t(k, idx):
    return 0.2 * s_k * np.sqrt(k[idx[0]][idx[1]][idx[2]])

# Get eddy mixing
# Input:
# k - turbulent energy field
# idx = (i,j,k) - 3D coordinates
def nu_m(k, idx):
    return 3.0 * c_e * nu_t(k, idx)

# Convert mixing ratio to mass fraction
# Input:
# q_a - mixing ratio
def x_a(q_a):
    return (q_a)/(q_a+1.0)

# Exner function to transform between potential and absolute temperature
# Input:
# p - pressure of humid air
def exnerFunc(p):
    return (p/p_G)**(gass_r, c_p)

# Get potential/absolute temperature from absolute/virtual temperature
# Input:
# T - absolute temperature
# groundT - temperature at ground level
# q_v - mixing ratio of vapor
# idx = (i,j,k) - 3D coordinate
def potentialTemp(T, groundT, q_v, idx):
    return T / exnerFunc(backgroundP(groundT, idx, q_v))

def absoluteTemp(theta, groundT, q_v, idx):
    return theta * exnerFunc(backgroundP(groundT, idx, q_v))

# Get potential virtual temperature
# Input:
# theta - potential tempterature
# q_v - mixing ratio of vapor
def virtualPotTemp(theta, q_v):
    return theta * (1+0.61*q_v)

# Compute buoyancy acceleration
# Input:
# groundT - temperature at ground level
# theta - potential temperature
# q - hydrometeor fields
# idx = (i,j,k) - 3D coordinates
def buoyancy(groundT, theta, q, idx):
    potentialT0 = potentialTemp(backgroundT, groundT, theta, idx)
    hydrometeors = np.sum(q[idx[0]][idx[1]][idx[2]][1:-1])
    b = theta/potentialT0 - 1 + 0.61*q[0] - hydrometeors
    return grav * b

# Get altitude from coordinate
# Input:
# Coordinate k
def altitude(k):
    return k * dx

# We use a MAC grid, this function obtains velocity values at the center by averaging the values at the walls
# Input:
# u, v, w - velocity fields
# idx - 3D index
def u_center(u, v, w, idx):
    return 0.5 * np.array([u[idx[0]][idx[1]][idx[2]]+u[idx[0]+1][idx[1]][idx[2]],
                           u[idx[0]][idx[1]][idx[2]]+u[idx[0]][idx[1]+1][idx[2]],
                           u[idx[0]][idx[1]][idx[2]]+u[idx[0]][idx[1]][idx[2]+1]])


# Helper computation for the dust-wind force interaction
# Input:
# u, v, w - velocity fields
# idx - 3D index
def C_dust(u, v, w, idx):

    # We need to compute the vector laplacian (i.e. the laplacian of each entry)
    nablaU = (u[idx[0]-1][idx[1]][idx[2]]+u[idx[0]+1][idx[1]][idx[2]]-2*u[idx[0]][idx[1]][idx[2]])/dx*dx
    + (u[idx[0]][idx[1]-1][idx[2]]+u[idx[0]][idx[1]+1][idx[2]]-2*u[idx[0]][idx[1]][idx[2]])/dx*dx
    + (u[idx[0]][idx[1]][idx[2]-1]+u[idx[0]][idx[1]][idx[2]+1]-2*u[idx[0]][idx[1]][idx[2]])/dx*dx

    nablaV = (v[idx[0]-1][idx[1]][idx[2]]+v[idx[0]+1][idx[1]][idx[2]]-2*v[idx[0]][idx[1]][idx[2]])/dx*dx
    + (v[idx[0]][idx[1]-1][idx[2]]+v[idx[0]][idx[1]+1][idx[2]]-2*v[idx[0]][idx[1]][idx[2]])/dx*dx
    + (v[idx[0]][idx[1]][idx[2]-1]+v[idx[0]][idx[1]][idx[2]+1]-2*v[idx[0]][idx[1]][idx[2]])/dx*dx
    
    nablaW = (w[idx[0]-1][idx[1]][idx[2]]+w[idx[0]+1][idx[1]][idx[2]]-2*w[idx[0]][idx[1]][idx[2]])/dx*dx
    + (w[idx[0]][idx[1]-1][idx[2]]+w[idx[0]][idx[1]+1][idx[2]]-2*w[idx[0]][idx[1]][idx[2]])/dx*dx
    + (w[idx[0]][idx[1]][idx[2]-1]+w[idx[0]][idx[1]][idx[2]+1]-2*w[idx[0]][idx[1]][idx[2]])/dx*dx

    norm = np.sqrt(nablaU*nablaU + nablaV * nablaV + nablaW * nablaW)
    Re = rho_dust * r_dust * norm / nu_dust
    c_dust = 1+Re/60.0+(Re/4.0)/(1+np.sqrt(Re))
    c_dust *= m_dust/(3*np.pi*r_dust*nu_dust)

    return c_dust

# Zero-moment reynolds stress for dust
# Input:
# \vector{u} - centered velocity field
# idx - 3D index
def tau_dust(u, idx):
    du_dz = (u[idx[0]][idx[1]][idx[2]+1]-u[idx[0]][idx[1]][idx[2]-1])/(2*dx)
    norm = np.sqrt(du_dz[0]*du_dz[0] + du_dz[1]*du_dz[1]+du_dz[2]*du_dz[2])
    z = altitude(idx[2])
    return rho_dust * norm * ((z**2)/c_k**2)

# Interaction force between dust and wind
def wind_dust_f(u, v, w, u_dust, v_dust, w_dust):
    return (rho_dust/C_dust) * np.array([u-u_dust, v-v_dust, w-w_dust])

# Prognosis equations for tropical cyclones (stress tensor, momentum, heat, and moisture)

# Input:
# \vector{u} = (u_rad, u_tan, u_z) - velocity field in cylindrical coordinates
def tauHurricane(u_rad, u_tan, u_z):
    norm = np.sqrt(u_rad*u_rad + u_tan*u_tan + u_z*u_z)
    return rho_d * norm * u_tan

# Input:
# \vector{u} = (u_rad, u_tan, u_z) - velocity field in cylindrical coordinates
# groundT - ground temperature
# idx - 3D index
def hHurricane(u_rad, u_tan, u_z, groundT, idx):
    norm = np.sqrt(u_rad*u_rad + u_tan*u_tan + u_z*u_z)
    return c_p * rho_d * norm * backgroundT(groundT, idx, gamma0, gamma1, z1)

# Input:
# theta - potential temperature
# \vector{u} = (u_rad, u_tan, u_z) - velocity field in cylindrical coordinates
def mHurricane(theta, u_rad, u_tan, u_z):
    norm = np.sqrt(u_rad*u_rad + u_tan*u_tan + u_z*u_z)
    return rho_d * norm * theta

# Input:
# theta - potential temperature
# groundT - ground temperature
# q - hydrometeor fields
# idx - 3D index
def qHurricane(theta, groundT, q, idx):
    return c_p * (absoluteTemp(theta, groundT, q[0], idx)-backgroundT(groundT, idx, gamma0, gamma1, z1))

# Holland's prognosis equation for dp_dr
# Input:
# r - radial coordinate
def dp_dr(r):
    holland = (R_max/r)**B
    return (B/r)*holland*np.exp(-holland)

